<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Group project - Political Media

---

Predict the message classes in the column `message` from the file `Political-media-DFE.csv`. Extract text data and include it into your modeling. 

To evaluate your model through cross-validation, set `cv` to the given cross-validation method.

The data stems from https://www.figure-eight.com/wp-content/uploads/2016/03/Political-media-DFE.csv.

In [36]:
import pandas as pd

In [84]:
df = pd.read_csv('Political-media-DFE.csv', encoding='latin')
pd.set_option('display.max_columns',100,'display.max_columns',100)
df

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,audience,audience:confidence,bias,bias:confidence,message,message:confidence,orig__golden,audience_gold,bias_gold,bioid,embed,id,label,message_gold,source,text
0,766192484,False,finalized,1,8/4/15 21:17,national,1.0,partisan,1.0000,policy,1.0,NaN,NaN,NaN,R000596,"<blockquote class=""twitter-tweet"" width=""450"">...",3.83249E+17,From: Trey Radel (Representative from Florida),NaN,twitter,RT @nowthisnews: Rep. Trey Radel (R- #FL) slam...
1,766192485,False,finalized,1,8/4/15 21:20,national,1.0,partisan,1.0000,attack,1.0,NaN,NaN,NaN,M000355,"<blockquote class=""twitter-tweet"" width=""450"">...",3.11208E+17,From: Mitch McConnell (Senator from Kentucky),NaN,twitter,VIDEO - #Obamacare: Full of Higher Costs and ...
2,766192486,False,finalized,1,8/4/15 21:14,national,1.0,neutral,1.0000,support,1.0,NaN,NaN,NaN,S001180,"<blockquote class=""twitter-tweet"" width=""450"">...",3.39069E+17,From: Kurt Schrader (Representative from Oregon),NaN,twitter,Please join me today in remembering our fallen...
3,766192487,False,finalized,1,8/4/15 21:08,national,1.0,neutral,1.0000,policy,1.0,NaN,NaN,NaN,C000880,"<blockquote class=""twitter-tweet"" width=""450"">...",2.98528E+17,From: Michael Crapo (Senator from Idaho),NaN,twitter,RT @SenatorLeahy: 1st step toward Senate debat...
4,766192488,False,finalized,1,8/4/15 21:26,national,1.0,partisan,1.0000,policy,1.0,NaN,NaN,NaN,U000038,"<blockquote class=""twitter-tweet"" width=""450"">...",4.07643E+17,From: Mark Udall (Senator from Colorado),NaN,twitter,.@amazon delivery #drones show need to update ...
5,766192489,False,finalized,2,8/4/15 21:06,national,1.0,partisan,0.5216,information,1.0,NaN,NaN,NaN,H001069,"<blockquote class=""twitter-tweet"" width=""450"">...",4.2103E+17,From: Heidi Heitkamp (Senator from North Dakota),NaN,twitter,Called on the @USDOTFRA to release info about ...
6,766192490,False,finalized,1,8/4/15 21:07,national,1.0,neutral,1.0000,mobilization,1.0,NaN,NaN,NaN,W000808,"<blockquote class=""twitter-tweet"" width=""450"">...",4.89407E+17,From: Frederica Wilson (Representative from Fl...,NaN,twitter,"@BBCWorld, help us keep the kidnapped Nigerian..."
7,766192491,False,finalized,1,8/4/15 20:51,constituency,1.0,neutral,1.0000,mobilization,1.0,NaN,NaN,NaN,B001279,"<blockquote class=""twitter-tweet"" width=""450"">...",3.27182E+17,From: Ron Barber (Representative from Arizona),NaN,twitter,Show your Arizona pride-choose your favorite S...
8,766192492,False,finalized,1,8/4/15 20:57,national,1.0,neutral,1.0000,personal,1.0,NaN,NaN,NaN,F000459,"<blockquote class=""twitter-tweet"" width=""450"">...",5.37292E+17,From: Chuck Fleischmann (Representative from T...,NaN,twitter,What a wonderful night at State Senator Ken Ya...
9,766192493,False,finalized,2,8/4/15 21:14,national,1.0,partisan,1.0000,support,1.0,NaN,NaN,NaN,H000874,"<blockquote class=""twitter-tweet"" width=""450"">...",2.98815E+17,From: Steny Hoyer (Representative from Maryland),NaN,twitter,Great op-ed by Pres. Clinton about signing #FM...


#### Use the following cross validation method

In [39]:
from sklearn.model_selection import StratifiedKFold

In [67]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

TypeError: __init__() got an unexpected keyword argument 'stratify'

In [41]:
df.isnull().sum()

_unit_id                  0
_golden                   0
_unit_state               0
_trusted_judgments        0
_last_judgment_at         0
audience                  0
audience:confidence       0
bias                      0
bias:confidence           0
message                   0
message:confidence        0
orig__golden           5000
audience_gold          5000
bias_gold              5000
bioid                     0
embed                     0
id                        0
label                     0
message_gold           5000
source                    0
text                      0
dtype: int64

In [42]:
df.message.value_counts()

policy          1411
personal        1170
support          921
information      647
media            277
attack           172
constituency     152
mobilization     129
other            121
Name: message, dtype: int64

In [60]:
df._golden.value_counts()

False    5000
Name: _golden, dtype: int64

In [45]:
df.rename(columns={"audience:confidence": "audience_confidence", "bias:confidence": "bias_confidence",
                  'message:confidence':'message_confidence', 'orig__golden': 'orig_golden'},inplace=True)

In [70]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegressionCV

model = LogisticRegressionCV(solver='lbfgs', multi_class='ovr', cv=kf)

gs_params = {'penalty': ['l1', 'l2'],
             'solver': ['liblinear'],
             'Cs': [np.logspace(-1, 1, 10)]}

gs = GridSearchCV(model, gs_params, cv=kf, n_jobs=2, verbose=1)

In [64]:
X = df[['audience','bias','source']]
y = df['message']

#Dummify
X = pd.get_dummies(X,drop_first=True)

#Scale
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [65]:
gs.fit(X,y)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:   17.7s finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
             error_score=nan,
             estimator=LogisticRegressionCV(Cs=10, class_weight=None,
                                            cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
                                            dual=False, fit_intercept=True,
                                            intercept_scaling=1.0,
                                            l1_ratios=None, max_iter=100,
                                            multi_class='ovr', n_jobs=None,
                                            penalty='l2', random_state=None,
                                            refit=True, scoring=None,
                                            solver='lbfgs', tol=0.0001,
                                            verbose=0),
             iid='deprecated', n_jobs=2,
             param_grid={'Cs': [array([ 0.1       ,  0.16681005,  0.27825594,  0.46415888,  0.77426368,
      

In [66]:
predictions = gs.predict(X)
print(classification_report(y, predictions))

              precision    recall  f1-score   support

      attack       0.00      0.00      0.00       172
constituency       0.00      0.00      0.00       152
 information       0.00      0.00      0.00       647
       media       0.00      0.00      0.00       277
mobilization       0.00      0.00      0.00       129
       other       0.00      0.00      0.00       121
    personal       0.29      0.92      0.44      1170
      policy       0.54      0.50      0.52      1411
     support       0.00      0.00      0.00       921

    accuracy                           0.36      5000
   macro avg       0.09      0.16      0.11      5000
weighted avg       0.22      0.36      0.25      5000



/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [77]:
df.text[3]

'RT @SenatorLeahy: 1st step toward Senate debate on Leahy-Crapo #VAWA bill is the Senate\x89Ûªs procedural vote today at 5:30 pm'

In [85]:
df['_last_judgment_at'] = pd.to_datetime(df['_last_judgment_at'])

In [87]:
df.dtypes

_unit_id                        int64
_golden                          bool
_unit_state                    object
_trusted_judgments              int64
_last_judgment_at      datetime64[ns]
audience                       object
audience:confidence           float64
bias                           object
bias:confidence               float64
message                        object
message:confidence            float64
orig__golden                  float64
audience_gold                 float64
bias_gold                     float64
bioid                          object
embed                          object
id                             object
label                          object
message_gold                  float64
source                         object
text                           object
dtype: object

In [89]:
df.source.unique()

array(['twitter', 'facebook'], dtype=object)